In [1]:
# Import the functions from /tools
import math
import os

# You need to choose one of the api's or hf's "request_llm" function here. 
# from tools.hf_open_source_llm_request import request_llm
# from tools.api_request import request_gpt as request_llm
from tools.api_request import request_gemini as request_llm

from tools.db_detail import db_getdesc
from tools.sql_execute import sqlite_execute as execute
from tqdm import tqdm
import json
import argparse

def sql_evoke(query,db_name):
    result, execution_time ,executable = execute("datasets/cosql_dataset/database/"+db_name+"/"+db_name+".sqlite",query)
    return result 

def get_example(db_name):
    sql_query = "SELECT name FROM sqlite_master WHERE type='table';"
    result = sql_evoke(sql_query,db_name)
    column_example=""
    for table_name in result:
        column_example = column_example + table_name[0] + ":\n"
        sql_get_eg = "SELECT * FROM "+ table_name[0] +" LIMIT 3;"
        table_eg = sql_evoke(sql_get_eg,db_name)
        for table_data in table_eg:
            column_example = column_example + '('
            for column_data in table_data: 
                column_example = column_example + str(column_data) +','
            column_example = column_example[:-1] + ')\n'
    return column_example

# print(db_getdesc("car_1"),get_example("car_1"))

In [2]:
from agents.rewriter import Rewriter
from agents.selector import Selector
from agents.detector import Detector
from agents.decomposer import Decomposer
from agents.refiner import Refiner

# creat and name the agents
rewriter = Rewriter("Rewriter")
selector = Selector("Selector")
detector = Detector("Detector")
decomposer = Decomposer("Decomposer")
refiner = Refiner("Refiner")

# input_data = "hey!"

# output_selector = selector.process_input(input_data)
# output_detector = detector.process_input(input_data)
# output_decomposer = decomposer.process_input(input_data)
# output_refiner = refiner.process_input(input_data)

In [ ]:
def process_json_part(data, output_file):
    for index1,item in enumerate(tqdm(data)):
        # Initialnize messages
        # print("__________"+str(index1)+"___________")
        db_name = item['db_name']
        previous_QA = ""
        for index, turn in enumerate(item['turns']):
            if turn['isuser']:
                
                user_question = turn['text']

                print("previous_QA " + ":" + previous_QA)
                print("question " + str(index) + ":" + user_question)
                
                if index+1<len(item['turns']):

                    print("________Question rewriter________")
                    input_data = {
                        "db_desc": db_getdesc(db_name),
                        "evidence": "",
                        "question": user_question,
                        "previous_QA": previous_QA
                    }
                    output_rewriter = rewriter.process_input(input_data)
                    
                    if output_rewriter["improper"] == "YES":
                        final_output = output_rewriter["text"]
                    else:
                        rewritten_question = output_rewriter["text"]
                        print("________Select columns and values________")
                        input_data = {
                            "db_name": db_name,
                            "db_desc": db_getdesc(db_name),
                            "db_exam": get_example(db_name),
                            "question": rewritten_question
                        }
                        
                        output_selector = selector.process_input(input_data)
                        
                        print("________Question type detect________")
                        input_data = {
                            "db_name": db_name,
                            "db_desc": db_getdesc(db_name),
                            "mini_schema": output_selector,
                            "question": rewritten_question
                        }
                        
                        output_detector = detector.process_input(input_data)
                        print(output_detector)

                        if output_detector == 'Yes':
                            print("________Decompose question and solve________")
        
                            print("________Refines erroneous SQL queries________")
                        
                        
                        
                        
                        else:
                            final_output = output_detector
                    # llm record
                    print("\nFINAL Response:")
                    print(final_output)
                    print("————————————————————————————————————————————————————————————")
                    item['turns'][index+1]['predict'] = final_output
                    # update messages
                    g_ans = ""
                    if item['turns'][index+1]['text']:
                        g_ans = item['turns'][index+1]['text']
                    else:
                        g_ans = item['turns'][index+1]['query']
                    previous_QA += "Q:" + user_question + "A:" + g_ans
                    
                    
        if not os.path.exists(output_file):
            with open(output_file, 'w') as f:
                items = [item]
                json.dump(items, f, indent=4)
                f.write('\n')
        else:
            with open(output_file, 'r') as f:
                try:
                    items = json.load(f)
                except json.JSONDecodeError:
                    print("\033[91mError:The file content is not in valid JSON format\033[0m")
                    # items = []

            if not isinstance(items, list):
                print("\033[91mError:The file content is not in valid JSON format\033[0m")
                # items = []

            items.append(item)

            with open(output_file, 'w') as f:
                json.dump(items, f, indent=4)
                f.write('\n') 

with  open('datasets/MMSQL_test.json', 'r', encoding='utf-8') as f:
        data = json.load(f)
    
process_json_part(data,'agenttest2.json')

  0%|                                                                                                                                     | 0/149 [00:00<?, ?it/s]

previous_QA :
question 0:What is the most popular car color?
________Question rewriter________
request gemini-1.5-flash
('Connection aborted.', ConnectionResetError(10054, '远程主机强迫关闭了一个现有的连接。', None, 10054, None))
('Connection aborted.', ConnectionResetError(10054, '远程主机强迫关闭了一个现有的连接。', None, 10054, None))
('Connection aborted.', ConnectionResetError(10054, '远程主机强迫关闭了一个现有的连接。', None, 10054, None))
('Connection aborted.', ConnectionResetError(10054, '远程主机强迫关闭了一个现有的连接。', None, 10054, None))
Success: Operation completed after 0 retries
{"improper": "YES", "rewritten": "What is the most popular car color?"}

FINAL Response:
What is the most popular car color?
————————————————————————————————————————————————————————————
previous_QA :Q:What is the most popular car color?A:This question cannot be answered because the provided database schema does not contain any information about car colors. Without a table or column that stores data on the color of cars, it is impossible to determine or list t